In [ ]:
# 1. Backend Web Development & API Design - Flask API with Prediction

# Install required libraries
%pip install Flask scikit-learn pandas numpy

import threading
from flask import Flask, jsonify, request
import sqlite3
import pandas as pd
from sklearn.linear_model import LinearRegression
import pickle

# Initialize the Flask app
app = Flask(__name__)

# SQLite DB setup
def init_db():
    conn = sqlite3.connect('example.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS predictions
                 (id INTEGER PRIMARY KEY, input_data TEXT, predicted_value REAL)''')
    conn.commit()
    conn.close()

# Model training (simulate training a simple model)
def train_model():
    data = {'feature1': [1, 2, 3, 4, 5],
            'feature2': [5, 4, 3, 2, 1],
            'target': [1, 2, 3, 4, 5]}
    df = pd.DataFrame(data)
    
    X = df[['feature1', 'feature2']]
    y = df['target']
    
    model = LinearRegression()
    model.fit(X, y)
    
    with open('model.pkl', 'wb') as f:
        pickle.dump(model, f)

# Load trained model
def load_model():
    with open('model.pkl', 'rb') as f:
        model = pickle.load(f)
    return model

# API to make predictions
@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        feature1 = data.get('feature1')
        feature2 = data.get('feature2')
        
        if feature1 is None or feature2 is None:
            return jsonify({'error': 'Missing required features'}), 400
        
        model = load_model()
        prediction = model.predict([[feature1, feature2]])[0]
        
        # Save prediction in the database
        conn = sqlite3.connect('example.db')
        c = conn.cursor()
        c.execute('''INSERT INTO predictions (input_data, predicted_value)
                     VALUES (?, ?)''', (str(data), prediction))
        conn.commit()
        conn.close()
        
        return jsonify({'prediction': prediction}), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Retrieve predictions from DB
@app.route('/predictions', methods=['GET'])
def get_predictions():
    conn = sqlite3.connect('example.db')
    c = conn.cursor()
    c.execute('SELECT * FROM predictions')
    rows = c.fetchall()
    conn.close()
    
    predictions = [{'id': row[0], 'input_data': row[1], 'predicted_value': row[2]} for row in rows]
    
    return jsonify(predictions), 200

# Start Flask in a separate thread to run in the background
def run_flask():
    app.run(debug=True, use_reloader=False, port=5000)

# Run Flask app in a background thread
threading.Thread(target=run_flask).start()


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\rrdas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
